In [10]:
import pandas as pd
import numpy as np

In [11]:
df1 = pd.read_csv("lawas\lawas-rainfall-daily.csv")
df1["DateTime"] = pd.to_datetime(df1["DateTime"],format="%Y-%m-%d")
df1.set_index("DateTime",inplace=True)

df2 = pd.read_csv("mulu\mulu-rainfall-daily.csv")
df2["DateTime"] = pd.to_datetime(df2["DateTime"],format="%Y-%m-%d")
df2.set_index("DateTime",inplace=True)

In [12]:
# def lagged_data(df):
#     for lag in range(1, 31):
#         df[f'Rainfall_lag_{lag}'] = df['Rainfall'].shift(lag)
#         df[f'Temperature_lag_{lag}'] = df['Temperature'].shift(lag)
#         df[f'Humidity_lag_{lag}'] = df['Humidity'].shift(lag)
#     return df

def feat_eng(df):
    """
    Prepare features for a single prediction date, now including forecasted features.
    """
    temp_df = df.copy()
        
    # Create interaction features
    source1 = ['Rainfall']
    source2 = ['TOTAL', 'ClimAdjust', 'ANOM']
    source3 = ['Temperature', 'DewPoint', 'Humidity', 'Visibility', 'WindSpeed', 'Pressure']
    source4 = ['Latitude', 'Longitude', 'Elevation']
    
    # Recreate all interaction features
    for s1 in source1:
        for s2 in source2:
            temp_df[f'{s1}_{s2}'] = temp_df[s1] * temp_df[s2]
        for s3 in source3:
            temp_df[f'{s1}_{s3}'] = temp_df[s1] * temp_df[s3]
    
    for s2 in source2:
        for s3 in source3:
            temp_df[f'{s2}_{s3}'] = temp_df[s2] * temp_df[s3]
    
    for i in range(len(source3)):
        for j in range(i + 1, len(source3)):
            temp_df[f'{source3[i]}_{source3[j]}'] = temp_df[source3[i]] * temp_df[source3[j]]
    
    for s4 in source4:
        for s1 in source1:
            temp_df[f'{s4}_{s1}'] = temp_df[s4] * temp_df[s1]
        for s2 in source2:
            temp_df[f'{s4}_{s2}'] = temp_df[s4] * temp_df[s2]
        for s3 in source3:
            temp_df[f'{s4}_{s3}'] = temp_df[s4] * temp_df[s3]
    
    temp_df['Rainfall_Temperature_Humidity'] = temp_df['Rainfall'] * temp_df['Temperature'] * temp_df['Humidity']
    
    # Create rolling statistics
    for window in [7, 14, 30]:
        temp_df[f"Rainfall_{window}d_mean"] = temp_df["Rainfall"].rolling(window).mean()
        temp_df[f"Rainfall_{window}d_std"] = temp_df["Rainfall"].rolling(window).std()
        temp_df[f"Rainfall_{window}d_sum"] = temp_df["Rainfall"].rolling(window).sum()
        temp_df[f"Rainfall_{window}d_min"] = temp_df["Rainfall"].rolling(window).min()
        temp_df[f"Rainfall_{window}d_max"] = temp_df["Rainfall"].rolling(window).max()
        temp_df[f"Rainfall_{window}d_median"] = temp_df["Rainfall"].rolling(window).median()
    

    temp_df['Day_of_Week'] = temp_df.index.dayofweek
    temp_df['Month'] = temp_df.index.month
    temp_df['Day_of_Year'] = temp_df.index.dayofyear
    
    # Encode cyclical features
    for col, max_val in [('Month', 12), ('Day_of_Week', 7), ('Day_of_Year', 366)]:
        temp_df[f'{col}_sin'] = np.sin(2 * np.pi * temp_df[col]/max_val)
        temp_df[f'{col}_cos'] = np.cos(2 * np.pi * temp_df[col]/max_val)
    
    # Drop original cyclical columns
    temp_df.drop(['Month', 'Day_of_Week', 'Day_of_Year'], axis=1, inplace=True)
    
    # Create lag features using the available data
    for lag in range(1, 31):
        temp_df[f'Rainfall_lag_{lag}'] = temp_df['Rainfall'].shift(lag)
        temp_df[f'Temperature_lag_{lag}'] = temp_df['Temperature'].shift(lag)
        temp_df[f'Humidity_lag_{lag}'] = temp_df['Humidity'].shift(lag)
    
    
    return temp_df

In [13]:
df1_fe = feat_eng(df1)
df2_fe = feat_eng(df2)

C:\Users\User\AppData\Local\Temp\ipykernel_21084\2769007660.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f'{col}_sin'] = np.sin(2 * np.pi * temp_df[col]/max_val)
C:\Users\User\AppData\Local\Temp\ipykernel_21084\2769007660.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f'{col}_cos'] = np.cos(2 * np.pi * temp_df[col]/max_val)
C:\Users\User\AppData\Local\Temp\ipykernel_21084\2769007660.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many 

In [14]:
result = pd.concat([df1_fe, df2_fe])

In [15]:
result.to_csv(f"lawas_mulu-rainfall-daily.csv",index=True)